# Exercise: Tool Calling

This notebook guides you through setting up and running a basic AI Agent using `LangChain`.
The steps include:

1. Installing required dependencies.
2. Importing necessary libraries.
3. Setting up the language model and the AI Agent.
4. Prompting the AI Agent.

Follow the instructions in the code cells and ensure all dependencies are installed correctly before proceeding.


In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required dependencies
!pip3 install llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
!pip3 install huggingface_hub==0.28.0
!pip3 install langchain==0.3.17 langchain-core==0.3.33 langchain-community==0.3.14

In [ ]:
# Import necessary libraries for downloading models and setting up the chat system
from random import randint
from huggingface_hub import hf_hub_download
from langchain.tools import tool
from langchain_community.chat_models import ChatLlamaCpp
from langchain_core.messages import HumanMessage

In [ ]:
# Download the LLM, you can search in Hugging Face
model_path = hf_hub_download(
    repo_id="Qwen/Qwen2.5-Coder-0.5B-Instruct-GGUF",
    filename="qwen2.5-coder-0.5b-instruct-q4_k_m.gguf",
    force_download=False,
)

In [ ]:
# Create the LLM
llm = ChatLlamaCpp(
    model_path=model_path,
    n_gpu_layers=25,
    stop=["<|im_end|>\n"],
    n_ctx=4096,
    max_tokens=4096,
    streaming=False,
    n_batch=256,
)

In [ ]:
# Create tools
@tool
def get_curr_temperature(city: str) -> int:
    """Get the current temperature of a city"""
    return randint(20, 30)

In [ ]:
# Bind the tools to the LLM
llm_tools = llm.bind_tools(
  [get_curr_temperature],
  tool_choice={"type": "function", "function": {"name": "get_curr_temperature"}},
)

In [ ]:
# Invoke the LLM with tools
messages = [
  HumanMessage("What is the current temperature in Madrid?")
]

all_tools_res = llm_tools.invoke(messages)

# We can append the LLM response to messages
# messages.append(all_tools_res)

In [ ]:
# Show the tool calls
all_tools_res.tool_calls

In [ ]:
# Call the tool
tool = all_tools_res.tool_calls[0]
tool_msg = get_curr_temperature.invoke(tool)

In [ ]:
# Add the tool response to the messages
tool_msg.additional_kwargs = {"args": tool["args"]}
messages.append(tool_msg)

In [ ]:
# Invoke the LLM with the new messages
llm.invoke(messages)